In [1]:
from transformers import AutoTokenizer

/Users/pingzhili/anaconda3/envs/ajax-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/gpt-oss-120b')

In [9]:
messages = [
    {"role": "user", "content": "How much is 1+1=?"},
    {"role": "assistant", "analysis": "1+1=2."}
]

In [45]:
messages = [
    {"role": "user", "content": "How much is 1+1=?"},
    {"role": "assistant", "thinking": "step1; step2; step3.", "content": "[CONTENT_PLACEHOLDER]"}
]

tokenizer.apply_chat_template(
    messages,
    # add_generation_prompt=True,
    reasoning_effort="low",
    tokenize=False,
    # continue_final_message=True
)

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-13\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>How much is 1+1=?<|end|><|start|>assistant<|channel|>analysis<|message|>step1; step2; step3.<|end|><|start|>assistant<|channel|>final<|message|>[CONTENT_PLACEHOLDER]<|return|>'

In [35]:
messages = [
    {"role": "user", "content": "How much is 1+1=?"},
]

tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    reasoning_effort="low",
    tokenize=False,
)

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-13\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>How much is 1+1=?<|end|><|start|>assistant'